## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import seaborn as sns

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import json

## Charger les données de Bordeaux

In [ ]:
# Chargement depuis parquet
df = pd.read_parquet("../data/clean/bordeaux_2022.parquet")

# Filtrer biens avec exactement 4 pièces principales
df_4p = df[df['Nombre pieces principales'] == 4].copy()

# 📤 Dictionnaires pour résultats sur Bordeaux
results_bordeaux = {"maison": [], "appartement": []}

## charger les données + modele utiliser sur Lille

In [ ]:
with open("resultats_modeles_maison_appartement.json", "r", encoding="utf-8") as f:
    results_lille = json.load(f)

## préparation des données immobilières

In [ ]:
colonnes_a_garder = [
    'Surface reelle bati',
    'Nombre pieces principales',
    'Nombre de lots',
    'Valeur fonciere',
    'Surface terrain',
    'Code type local',
]

# logements
df_logements = df_4p[colonnes_a_garder].copy()

#Calcul du prix au m² pondéré (surface bâtie + 0.3 * surface terrain) 
df_logements['prix_m2'] = df_logements['Valeur fonciere'] / df_logements['Surface reelle bati'] + (0.3 * df_logements['Surface terrain'].fillna(0))

## Nettoyage et filtrage des données immobilières

In [ ]:
# Supprimer les valeurs aberrantes communes
df_logements = df_logements[
    (df_logements["Valeur fonciere"] > 500) &
    (df_logements["Valeur fonciere"] < 1_500_000) &
    (df_logements["Surface reelle bati"] >= 20) &
    (df_logements["Surface reelle bati"] <= 200) &
    (df_logements["prix_m2"] > 500) &
    (df_logements["prix_m2"] < 10_000)
]

# Appliquer les filtres spécifiques aux maisons uniquement
is_maison = df_logements["Code type local"] == 1
df_logements = df_logements[
    ~is_maison | (  # On garde tout sauf les maisons, OU les maisons avec terrain entre 100 et 1500 m²
        (df_logements["Surface terrain"] >= 100) & (df_logements["Surface terrain"] <= 1500)
    )
]

# Remplacer Surface terrain NaN par 0 pour les appartements (Code type local == 1)
df_logements.loc[
    (df_logements["Code type local"] == 2) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0

#séparer les données
df_bx_apt = df_logements[df_logements["Code type local"] == 2].copy()
df_bx_maisons = df_logements[df_logements["Code type local"] == 1].copy()

## Préparer les données pour l'entraînement

In [ ]:
# Variables explicatives
features = ["Valeur fonciere", "Surface terrain", "Surface reelle bati"]

#maison
X_bx_maisons = df_bx_apt[features]
y_bx_maisons = df_bx_apt["prix_m2"]

# appartement
X_bx_apt = df_bx_maisons[features]
y_bx_apt = df_bx_maisons["prix_m2"]

## import des modeles

In [ ]:
# 🔧 Liste des modèles
model_names = [
    "Linear Regression",
    "Decision Tree (base)",
    "Decision Tree (grid)",
    "Random Forest (base)",
    "Random Forest (grid)",
    "XGBoost"
]

# 🔁 Tester tous les modèles d’un type de logement sur Bordeaux
def evaluate_models_on_bordeaux(type_logement, X_bordeaux, y_bordeaux):
    for model_name in model_names:
        # Nettoyage nom de fichier
        filename = f"../models/{type_logement}/modele_{type_logement}_{model_name}.joblib"
        model = joblib.load(filename)

        # Prédictions + métriques
        y_pred = model.predict(X_bordeaux)
        mse = mean_squared_error(y_bordeaux, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_bordeaux, y_pred)
        r2 = r2_score(y_bordeaux, y_pred)

        results_bordeaux[type_logement].append({
            "Modèle": model_name,
            "MSE": mse,
            "RMSE": rmse,
            "MAE": mae,
            "R2 Score": r2
        })

# 🏘️ Évaluer pour maisons et appartements
evaluate_models_on_bordeaux("maison", X_bx_maisons, y_bx_maisons)
evaluate_models_on_bordeaux("appartement", X_bx_apt, y_bx_apt)

## Prédiction et évaluation (MSE, RMSE, MAE, R²)

In [ ]:
# Comparaison avec 2 tableaux
def compare_results(type_logement):
    print(f"\n📊 Comparaison pour {type_logement.title()}:\n")

    df_lille = pd.DataFrame(results_lille[type_logement])
    df_bordeaux = pd.DataFrame(results_bordeaux[type_logement])
    df = df_lille.merge(df_bordeaux, on="Modèle", suffixes=("_Lille", "_Bordeaux"))

    # Tableau 1 : Scores
    base_cols = ["Modèle"]
    for metric in ["MSE", "RMSE", "MAE", "R2 Score"]:
        base_cols.extend([f"{metric}_Lille", f"{metric}_Bordeaux"])
    df_base = df[base_cols].round(2)

    # Tableau 2 : Différences
    diff_cols = ["Modèle"]
    for metric in ["MSE", "RMSE", "MAE", "R2 Score"]:
        df[f"{metric}_Diff"] = df[f"{metric}_Bordeaux"] - df[f"{metric}_Lille"]
        df[f"{metric}_Pct"] = 100 * df[f"{metric}_Diff"] / df[f"{metric}_Lille"]
        diff_cols.extend([f"{metric}_Diff", f"{metric}_Pct"])
    df_diff = df[diff_cols].round(2)

    # Affichage
    print("🔹 Tableau 1 : Scores Lille vs Bordeaux")
    print(df_base)
    print("\n🔸 Tableau 2 : Écarts et pourcentages (%)")
    print(df_diff)

    # Optionnel : Sauvegarde CSV
    os.makedirs(f"comparaison/{type_logement}", exist_ok=True)
    df_base.to_csv(f"comparaison/{type_logement}/comparaison_{type_logement}_scores.csv", index=False)
    df_diff.to_csv(f"comparaison/{type_logement}/comparaison_{type_logement}_diff.csv", index=False)

# Appliquer
compare_results("maison")
compare_results("appartement")

## 🏡 Analyse comparative des performances des modèles — Lille vs Bordeaux

### 🏘️ Maisons

#### ✅ Performances à Lille :
|Modèle|	MSE|	RMSE|	MAE|	R² Score
|---|---|---|---|---
|XGBoost|	34 158|	184.82|	146.83|	0.9502
|Decision Tree (base)|	61 008|	246.99|	177.05|	0.9111
|Linear Regression|	88 679|	297.79|	236.67|	0.8708
|Random Forest (base)|	98 844|	314.39|	179.26|	0.8560

➡️ Meilleur modèle à Lille : XGBoost, avec les meilleurs scores sur tous les indicateurs.

#### 🔄 Comparaison à Bordeaux (écarts) :
|Modèle|	Δ MSE (%)|	Δ RMSE (%)|	Δ MAE (%)|	Δ R² (%)
|---|---|---|---|---
|XGBoost|	+233.7%|	+82.7%|	+22.3%|	−1.0%
|Decision Tree (base)|	+330.7%|	+107.5%|	+48.5%|	−5.2%
|Linear Regression|	+40.4%|	+18.5%|	−13.2%|	+7.4% ✅
|Random Forest (base)|	+16.9%|	+8.1%|	−11.6%|	+9.8% ✅

➡️ Meilleur modèle à Bordeaux : Random Forest (base)
Il présente la plus faible dégradation des performances en MSE, RMSE et MAE, tout en gagnant du R².

#### 🏁 Modèle global retenu pour les maisons :
|Jeu|	Modèle retenu|	Justification|
|---|---|---
|Lille|	XGBoost	Meilleures| perfs globales
|Bordeaux|	Random Forest (base)|	Meilleure stabilité, gains sur MAE et R²
|Global|	Random Forest (base)|	Compromis performance/généralisation

### 🏢 Appartements

#### ✅ Performances à Lille :
|Modèle|	MSE|	RMSE|	MAE|	R² Score
|---|---|---|---|---
|Decision Tree (base)|	46 346|	215.28|	150.61|	0.9670
|Random Forest (base)|	50 360|	224.41|	107.75|	0.9642
|XGBoost|	71 256|	266.94|	121.87|	0.9493
|Linear Regression|	120 882|	347.68|	185.99|	0.9140

➡️ Meilleur modèle à Lille : Decision Tree, très bon compromis sur tous les scores.

#### 🔄 Comparaison à Bordeaux (écarts) :
|Modèle|	Δ MSE (%)|	Δ RMSE (%)|	Δ MAE (%)|	Δ R² (%)
|---|---|---|---|---
|Linear Regression|	+176.8%|	+66.4%|	+118.0%|	−7.6% ✅
|Decision Tree (base)|	+1524%|	+303.1%|	+263.1%|	−32.8%
|Random Forest (base)|	+943.9%|	+223.1%|	+442.1%|	−21.7%
|XGBoost|	+751.8%|	+191.9%|	+361.8%|	−24.4%

➡️ Meilleur modèle à Bordeaux : Linear Regression
Malgré de faibles performances à Lille, il s’avère être le plus stable et le moins dégradé à Bordeaux.

#### 🏁 Modèle global retenu pour les appartements :
|Jeu|	Modèle retenu|	Justification
|---|---|---
|Lille|	Decision Tree|	Meilleurs scores
|Bordeaux|	Linear Regression|	Moins sensible à la généralisation
|Global|	Linear Regression|	Meilleur compromis si déploiement multi-ville

### 🧠 Conclusion globale

- Les modèles complexes (XGBoost, Random Forest) performent bien sur les données d'entraînement mais peuvent perdre fortement en généralisation.
- Les modèles simples (Linear Regression) sont parfois plus robustes sur des jeux de données différents, bien qu’ils soient moins précis initialement.
- Pour un déploiement dans plusieurs villes, Random Forest pour les maisons et Linear Regression pour les appartements semblent les meilleurs choix globaux.



## 🧪 6. Comparaison des performances entre Lille et Bordeaux pour chaque type de logement


### 🔍 Le modèle est-il aussi performant sur Bordeaux ?

#### 🏘️ Maisons

Non, les performances du modèle XGBoost chutent significativement sur Bordeaux, malgré d'excellents résultats à Lille. En revanche, le modèle Random Forest (base), bien que moins performant à Lille, se révèle plus stable et efficace à Bordeaux.

📉 Écarts de performance observés :
- XGBoost (Maisons) :
    - 📈 MSE : +233.7 %
    - 📈 RMSE : +82.7 %
    - 📈 MAE : +22.3 %
    - 📉 R² : −1.0 point
- Random Forest (base) :
    - 📈 MSE : +16.9 %
    - 📈 RMSE : +8.1 %
    - 📉 MAE : −11.6 %
    - 📈 R² : +9.8 points

Le modèle Random Forest, bien que "moins bon" à Lille, généralise mieux sur Bordeaux.

#### 🏢 Appartements

Non, les modèles performants à Lille (comme le Decision Tree) se dégradent fortement sur Bordeaux. À l’inverse, le modèle Linear Regression, médiocre à Lille, devient le plus stable sur Bordeaux.

📉 Écarts de performance observés :
- Decision Tree (Appartements) :
    - 📈 MSE : +1524 %
    - 📈 RMSE : +303.1 %
    - 📈 MAE : +263.1 %
    - 📉 R² : −32.8 points
- Linear Regression :
    - 📈 MSE : +176.8 %
    - 📈 RMSE : +66.4 %
    - 📈 MAE : +118.0 %
    - 📉 R² : −7.6 points

Cela montre que le modèle Decision Tree est très spécifique au jeu d'entraînement (overfitting possible), tandis que Linear Regression reste plus robuste même s'il est moins précis à l’origine.

### 🧠 Quels facteurs peuvent expliquer ces différences ?
1. Volume ou diversité des données :
- Il est possible que les données de Lille soient plus homogènes que celles de Bordeaux.
- Le manque de données représentatives pour Bordeaux peut entraîner une mauvaise généralisation.
2. Complexité du modèle :
- Les modèles comme XGBoost ou Decision Tree sont sensibles au sur-apprentissage s’ils captent trop de détails spécifiques au jeu de départ.
3. Type de bien :
- Il semble que les maisons soient plus prévisibles (meilleure généralisation avec Random Forest).
- Les appartements montrent une plus grande hétérogénéité et demandent peut-être des modèles plus simples.

### ✅ Le modèle généralise-t-il mieux sur un type de bien que sur l'autre ?
Oui, les modèles généralisent globalement mieux sur les maisons que sur les appartements.
- Pour les maisons, le modèle Random Forest montre une bonne stabilité des scores entre Lille et Bordeaux.
- Pour les appartements, les modèles plus complexes se dégradent fortement hors entraînement, sauf pour la régression linéaire qui reste la plus stable malgré sa moindre précision.